In [1]:
import os
import pandas as pd
import numpy as np
import imageio
from PIL import Image
import cv2
import random


In [2]:
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
# from keras.optimizers import SGD, RMSprop, adam
from keras.utils import np_utils
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split 
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2

from PIL import Image
# import theano

In [3]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [156]:
from PIL import Image

im = Image.open("Test/Normal/d_1063.png")
im.show()

In [48]:
test_images = []
IMG_SIZE = 28
for images in os.listdir("Test/Normal"):
    img_array = cv2.imread('Test/Normal/'+images)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    test_images.append([new_array,0])

for images in os.listdir("Test/Reversal"):
    img_array = cv2.imread('Test/Reversal/'+images)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    test_images.append([new_array,1])

In [49]:
train_images = []
IMG_SIZE = 28
for images in os.listdir("Train/Normal"):
    img_array = cv2.imread('Train/Normal/'+images)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    train_images.append([new_array,0])

for images in os.listdir("Train/Reversal"):
    img_array = cv2.imread('Train/Reversal/'+images)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    train_images.append([new_array,1])

In [50]:
random.shuffle(train_images)
random.shuffle(test_images)

In [51]:
X_train =[]
y_train =[]
#Normal- 0, Reversed - 1
#features - img, label - 0/1
#flattening
for features, label in train_images:
    X_train.append(features)
    y_train.append(label)
X_train = np.array(X_train).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [52]:
X_test =[]
y_test =[]
for features, label in test_images:
    X_test.append(features)
    y_test.append(label)
X_test = np.array(X_test).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [53]:
X_train = X_train.astype('float32')
X_train /= 255
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train, 2)
print(y_train[100])
print(y_train.shape)

[0. 1.]
(86115, 2)


In [54]:
X_test = X_test.astype('float32')
X_test /= 255
from keras.utils import np_utils
y_test= np_utils.to_categorical(y_test, 2)
print(y_test[100])
print(y_test.shape)

[0. 1.]
(37439, 2)


In [55]:
batch_size = 16
nb_classes =2
nb_epochs = 5
img_rows, img_columns = 28,28
img_channel = 3
nb_filters = 32
nb_pool = 2
nb_conv = 3

In [148]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu,
                           input_shape=(28, 28, 3)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(1,  activation=tf.nn.sigmoid)
])

In [149]:
model.compile(optimizer='adam', loss='mean_squared_error',metrics=['accuracy'])

In [112]:
# print(X_test[0].shape)
c = tf.reshape(y_train[0], [-1])
print(c.shape)
# y_train[0].shape

(2,)


In [150]:
model.fit(X_train, y_train, batch_size = batch_size, epochs = nb_epochs, verbose = 1, validation_data = (X_test, y_test))

Epoch 1/5
4653/5383 [========================>.....] - ETA: 1:12 - loss: 0.2500 - accuracy: 0.5000

KeyboardInterrupt: 

In [114]:
model.save('dyslexia_model')

INFO:tensorflow:Assets written to: dyslexia_model\assets


INFO:tensorflow:Assets written to: dyslexia_model\assets


In [115]:
from tensorflow import keras
model1 = keras.models.load_model('dyslexia_model')

In [116]:
model1.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_36 (Conv2D)          (None, 28, 28, 16)        448       
                                                                 
 max_pooling2d_36 (MaxPoolin  (None, 14, 14, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_37 (Conv2D)          (None, 14, 14, 16)        2320      
                                                                 
 max_pooling2d_37 (MaxPoolin  (None, 7, 7, 16)         0         
 g2D)                                                            
                                                                 
 flatten_22 (Flatten)        (None, 784)               0         
                                                                 
 dropout_17 (Dropout)        (None, 784)             

In [155]:
model1.predict(np.array(X_test[550:551]))

1/1 [==============================] - 0s 65ms/step


array([[0.9951997 , 0.00480034]], dtype=float32)

In [138]:
type(X_train[0])

numpy.ndarray

In [142]:
loss, acc = model1.evaluate(X_test, y_test, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

1170/1170 - 25s - loss: 0.1528 - accuracy: 0.8322 - 25s/epoch - 21ms/step
Restored model, accuracy: 83.22%


In [195]:
#Inference
# im = Image.open("Test/Reversal/1_32039.png")
img_array = cv2.imread('Train/Reversal/1_31975.png')
new_array = cv2.resize(img_array, (28, 28))
print(type(img_array))

<class 'numpy.ndarray'>


In [196]:
temp = []
temp.append(img_array)
a = np.array(temp)

In [198]:
model1.predict(a)

1/1 [==============================] - 0s 73ms/step


array([[0., 1.]], dtype=float32)

In [5]:
df_test = pd.DataFrame(test_images,columns=['image_vector','label'])
df_test.head()
len(df_test)

37439

In [6]:
train_images = []
for images in os.listdir("Train/Normal"):
    image = Image.open('Train/Normal/'+images).convert('L')
    # Convert the image to a numpy array
    image_array = np.array(image)
    image_vector = image_array.flatten()
    train_images.append([image_vector,0])

for images in os.listdir("Train/Reversal"):
    image = Image.open('Train/Reversal/'+images).convert('L')
    # Convert the image to a numpy array
    image_array = np.array(image)
    image_vector = image_array.flatten()
    train_images.append([image_vector,1])

In [7]:
df_train = pd.DataFrame(train_images,columns=['image_vector','label'])
df_train.head()
len(df_train)

86115

In [8]:
df_train = df_train.sample(frac = 1).reset_index(drop=True)

In [9]:
df_train

,image_vector,label
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
...,...,...
86110,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
86111,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
86112,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
86113,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1


In [52]:
!pip install tensorflow

  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
